<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/Tensorflow%20Lite/Tensorflow%20Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and setting up the environment

In [1]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 60kB/s 
     |████████████████████████████████| 368kB 47.4MB/s 
     |████████████████████████████████| 3.2MB 47.6MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


## Stage 2: Importing project dependencies

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.1'

## Stage 3: Dataset preprocessing

### Loading the FashionMNIST dataset

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Image normalization

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

### Reshaping of the dataset

In [5]:
X_train.shape

(60000, 28, 28)

In [6]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

## Stage 4: Building a model

### Defining the model

In [7]:
model = tf.keras.models.Sequential()

### Building the model

In [8]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compiling the model

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Training the model

In [10]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 104us/sample - loss: 0.5295 - sparse_categorical_accuracy: 0.8128
Epoch 2/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3990 - sparse_categorical_accuracy: 0.8563
Epoch 3/5
60000/60000 [==============================] - 6s 106us/sample - loss: 0.3678 - sparse_categorical_accuracy: 0.8669
Epoch 4/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.3463 - sparse_categorical_accuracy: 0.8717
Epoch 5/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.3310 - sparse_categorical_accuracy: 0.8771


### Evaluating the model

In [11]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 0s 41us/sample - loss: 0.3805 - sparse_categorical_accuracy: 0.8603
Test accuracy: 0.8603000044822693


## Stage 5: Converting the model into TensorFlow Lite version

### Saving the model

In [12]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Creating the TFLite Converter

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Converting the model

In [14]:
tflite_model = converter.convert()

### Saving the TFLite version of the model

In [15]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)